In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys # import sys to get more detailed Python exception info
import psycopg2 # import the connect library for psycopg2
from psycopg2 import OperationalError, errorcodes, errors # import the error handling libraries for psycopg2
import psycopg2.extras as extras
import pandas as pd
import glob

In [3]:
path = r'/home/jovyan/work/input_dataset'
all_files = glob.glob(path + "/*")

li = []

for filename in all_files:
    print("Merging: ",filename)
    df = pd.read_csv(filename, index_col=None, header=0,sep='\t')
    li.append(df)

tradeData = pd.concat(li, axis=0, ignore_index=True)        

Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200901
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200902
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200903
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200904
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200905
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200906
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200907
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200908
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200909
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200910
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200911
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200912
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200913
Merging:  /home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-2

In [4]:
#tradeData = pd.read_csv('/home/jovyan/work/input_dataset/HB-Spot-Tick-BCHUSDT-20200901',sep='\t')
tradeData.head()

,tradeid,symbol,ts,price,amount,direction
0,101089750351,bchusdt,1598889601108,277.50,0.9941,sell
1,101089750352,bchusdt,1598889601260,277.44,0.2000,buy
2,101089750353,bchusdt,1598889601260,277.44,1.0000,buy
3,101089750354,bchusdt,1598889601260,277.44,1.0000,buy
4,101089750355,bchusdt,1598889601579,277.45,0.6770,sell


In [5]:
tradeData['ts'] = tradeData['ts'].apply(lambda x: pd.datetime.fromtimestamp(x / 1000))
tradeData['ts']

<ipython-input-5-3bcfa6a90b93>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  tradeData['ts'] = tradeData['ts'].apply(lambda x: pd.datetime.fromtimestamp(x / 1000))


0         2020-08-31 16:00:01.108
1         2020-08-31 16:00:01.260
2         2020-08-31 16:00:01.260
3         2020-08-31 16:00:01.260
4         2020-08-31 16:00:01.579
                    ...          
6135465   2020-09-30 15:59:07.755
6135466   2020-09-30 15:59:10.664
6135467   2020-09-30 15:59:13.146
6135468   2020-09-30 15:59:43.181
6135469   2020-09-30 15:59:43.181
Name: ts, Length: 6135470, dtype: datetime64[ns]

In [6]:
print("Min date:",tradeData.groupby('symbol')['ts'].min())
print("Max date:",tradeData.groupby('symbol')['ts'].max())

Min date: 2020-08-31 16:00:00.820000
Max date: 2020-09-30 15:59:44.491000


In [7]:
# Note: please change your database, username & password as per your own values
conn_params_dic = {
    "host"      : "postgres-source",
    "database"  : "db",
    "user"      : "root",
    "password"  : "toor"
}

In [8]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    
    
# Define a connect function for PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successfully..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

def init_table(conn):
    cursor = conn.cursor()
    cursor.execute('''
DROP TABLE IF EXISTS trade ;

CREATE TABLE trade
(
    id bigint NOT NULL,
    symbol text,
    ts TIMESTAMP WITHOUT TIME ZONE ,
    price double precision,
    amount double precision,
    direction text,
    CONSTRAINT trade_pkey PRIMARY KEY (id)
)''')


# Define function using copy_from_dataFile to insert the dataframe.
def copy_from_dataFile(conn, df, table):
    
#  Here we are going save the dataframe on disk as a csv file, load # the csv file and use copy_from() to copy it to the table
    tmp_df = "./df_temp.csv"
    df.to_csv(tmp_df, header=False,index = False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
        print("Data inserted using copy_from_datafile() successfully....")
        cursor.close()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()
    os.remove(tmp_df)    

In [9]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
#init table
init_table(conn)
# Run the copy_from_dataFile method, here saving data into 
copy_from_dataFile(conn, tradeData, 'trade')
# Close the connection
#conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using copy_from_datafile() successfully....


In [10]:
# Prepare sql query
sql = "SELECT * FROM trade" 
cursor = conn.cursor()
# Execute cursor
cursor.execute(sql)
# Fetch all the records
tuples = cursor.fetchall()
# list of columns
cols = list(tradeData.columns)
tradedf = pd.DataFrame(tuples,columns=cols) 
# Print few records
print(tradedf.head())

# Close the cursor
cursor.close()
# Close the connection
conn.close()

        tradeid   symbol                      ts   price  amount direction
0  101089750351  bchusdt 2020-08-31 16:00:01.108  277.50  0.9941      sell
1  101089750352  bchusdt 2020-08-31 16:00:01.260  277.44  0.2000       buy
2  101089750353  bchusdt 2020-08-31 16:00:01.260  277.44  1.0000       buy
3  101089750354  bchusdt 2020-08-31 16:00:01.260  277.44  1.0000       buy
4  101089750355  bchusdt 2020-08-31 16:00:01.579  277.45  0.6770      sell


In [11]:
tradedf.equals(tradeData)

True

In [12]:
tradedf

,tradeid,symbol,ts,price,amount,direction
0,101089750351,bchusdt,2020-08-31 16:00:01.108,277.50,0.9941,sell
1,101089750352,bchusdt,2020-08-31 16:00:01.260,277.44,0.2000,buy
2,101089750353,bchusdt,2020-08-31 16:00:01.260,277.44,1.0000,buy
3,101089750354,bchusdt,2020-08-31 16:00:01.260,277.44,1.0000,buy
4,101089750355,bchusdt,2020-08-31 16:00:01.579,277.45,0.6770,sell
...,...,...,...,...,...,...
6135465,101406166387,ltcusdt,2020-09-30 15:59:07.755,46.24,1.3312,buy
6135466,101406166388,ltcusdt,2020-09-30 15:59:10.664,46.24,1.2709,buy
6135467,101406166389,ltcusdt,2020-09-30 15:59:13.146,46.24,2.3396,buy
6135468,101406166390,ltcusdt,2020-09-30 15:59:43.181,46.23,1.7302,sell


In [13]:
tradedf['symbol'].unique()

array(['bchusdt', 'etcusdt', 'ltcusdt'], dtype=object)

In [17]:
conn_params_dic_sink = {
    "host"      : "postgres-sink",
    "database"  : "db",
    "user"      : "root",
    "password"  : "toor"
}

# Prepare sql query
sql = "SELECT COUNT(*) FROM trade" 
conn_sink = connect(conn_params_dic_sink)
cursor = conn_sink.cursor()
# Execute cursor
cursor.execute(sql)
print(cursor.fetchall())

Connecting to the PostgreSQL...........
Connection successfully..................
[(1787270,)]
